In [2]:
# !pip install psycopg2

In [1]:
import pandas as pd
import psycopg2
# Use psycopg2's execute_values for efficient bulk insert
from psycopg2.extras import execute_values
from psycopg2 import sql

In [2]:
# Database connection parameters (replace with actual credentials)
db_params = {
    'dbname': 'smog_db',
    'user': 'smog_dev',
    'password': 'kcSzXTAg1RSXktipRM9gN9cD32pGFN2k',
    'host': 'dpg-csnu6npu0jms73913fh0-a.oregon-postgres.render.com',
    'port': '5432'
}

In [3]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [4]:
# Function to list all tables
def list_tables():
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

In [5]:
list_tables()

Tables in the database:
combined_hourly_data


In [6]:
# Read the CSV file
file_path = 'combined_hourly_data.csv'
data = pd.read_csv(file_path)


In [7]:
# Pivot the data to have unique districts as columns
pivot_data = data.pivot_table(index=['date', 'hour'], columns='District', values='AQI', fill_value=None)
pivot_data.head(6)

District           Attock  Bahawalnagar  Bahawalpur  Bhakkar  Chakwal  \
date       hour                                                         
11/15/2024 10:00   89.590        74.700      99.855  180.090  116.535   
           11:00   91.975        76.960      92.260  178.455  116.890   
           12:00   99.530       122.645     117.070  182.415  139.250   
           13:00  100.655       136.335     139.365  171.570  139.060   
           14:00   98.000       152.590     167.385  150.310  138.465   
           15:00   96.890       174.255     187.155  122.090  134.765   

District          Chiniot  Dera Ghazi Khan  Faisalabad  Gujranwala   Gujrat  \
date       hour                                                               
11/15/2024 10:00  189.375          174.945     159.635     161.510  135.750   
           11:00  198.205          161.255     194.490     165.470  134.815   
           12:00  241.485          172.855     256.750     188.555  157.875   
           13:00  289.395          178.560     296.360     207.560  172.225   
           14:00  330.465          190.330     318.400     221.225  182.335   
           15:00  335.950          202.285     315.140     226.480  184.380   

District          ...  Pakpattan  Rahim Yar Khan  Rajanpur  Rawalpindi  \
date       hour   ...                                                    
11/15/2024 10:00  ...     78.500          93.115   112.065     145.925   
           11:00  ...     87.215          88.845   103.535     220.875   
           12:00  ...    144.225         111.275   118.115     293.135   
           13:00  ...    182.005         126.160   128.670     310.255   
           14:00  ...    227.145         148.090   148.865     301.445   
           15:00  ...    258.950         166.880   166.320     275.610   

District          Sahiwal  Sargodha  Sheikhupura  Sialkot  Toba Tek Singh  \
date       hour                                                             
11/15/2024 10:00   83.545   193.815      175.915  164.685         144.170   
           11:00   86.485   198.395      196.615  171.610         148.520   
           12:00  138.675   226.665      252.250  186.835         191.110   
           13:00  164.870   214.125      286.820  204.770         227.325   
           14:00  198.145   193.625      318.470  217.380         264.895   
           15:00  221.650   189.280      328.955  222.620         254.140   

District           Vehari  
date       hour            
11/15/2024 10:00   87.560  
           11:00   89.740  
           12:00  125.900  
           13:00  159.690  
           14:00  204.605  
           15:00  236.910  

[6 rows x 36 columns]

In [8]:
# Reset the index to flatten the DataFrame
pivot_data.reset_index(inplace=True)

In [9]:
pivot_data.head(6)

District,date,hour,Attock,Bahawalnagar,Bahawalpur,Bhakkar,Chakwal,Chiniot,Dera Ghazi Khan,Faisalabad,...,Pakpattan,Rahim Yar Khan,Rajanpur,Rawalpindi,Sahiwal,Sargodha,Sheikhupura,Sialkot,Toba Tek Singh,Vehari
0,11/15/2024,10:00,89.590,74.700,99.855,180.090,116.535,189.375,174.945,159.635,...,78.500,93.115,112.065,145.925,83.545,193.815,175.915,164.685,144.170,87.560
1,11/15/2024,11:00,91.975,76.960,92.260,178.455,116.890,198.205,161.255,194.490,...,87.215,88.845,103.535,220.875,86.485,198.395,196.615,171.610,148.520,89.740
2,11/15/2024,12:00,99.530,122.645,117.070,182.415,139.250,241.485,172.855,256.750,...,144.225,111.275,118.115,293.135,138.675,226.665,252.250,186.835,191.110,125.900
3,11/15/2024,13:00,100.655,136.335,139.365,171.570,139.060,289.395,178.560,296.360,...,182.005,126.160,128.670,310.255,164.870,214.125,286.820,204.770,227.325,159.690
4,11/15/2024,14:00,98.000,152.590,167.385,150.310,138.465,330.465,190.330,318.400,...,227.145,148.090,148.865,301.445,198.145,193.625,318.470,217.380,264.895,204.605
5,11/15/2024,15:00,96.890,174.255,187.155,122.090,134.765,335.950,202.285,315.140,...,258.950,166.880,166.320,275.610,221.650,189.280,328.955,222.620,254.140,236.910


In [10]:
# Get the list of columns (date, hour, and unique district names)
columns = pivot_data.columns.tolist()
columns

['date',
 'hour',
 'Attock',
 'Bahawalnagar',
 'Bahawalpur',
 'Bhakkar',
 'Chakwal',
 'Chiniot',
 'Dera Ghazi Khan',
 'Faisalabad',
 'Gujranwala',
 'Gujrat',
 'Hafizabad',
 'Jhang',
 'Jhelum',
 'Kasur',
 'Khanewal',
 'Khushab',
 'Lahore',
 'Layyah',
 'Lodhran',
 'Mandi Bahuddin',
 'Mianwali',
 'Multan',
 'Muzaffargarh',
 'Nankana Sahib',
 'Narowal',
 'Okara',
 'Pakpattan',
 'Rahim Yar Khan',
 'Rajanpur',
 'Rawalpindi',
 'Sahiwal',
 'Sargodha',
 'Sheikhupura',
 'Sialkot',
 'Toba Tek Singh',
 'Vehari']

In [11]:
# Create table with date, hour, and unique district columns
create_table_query = sql.SQL(
    "CREATE TABLE IF NOT EXISTS combined_hourly_data ("
    "date DATE, "
    "hour INTEGER, "
    + ", ".join([f'"{col}" FLOAT' for col in columns[2:]])
    + ");"
)

# Execute the query to create the table
cursor.execute(create_table_query)


In [12]:
# Alter the table to change the hour column to TIME with explicit casting
alter_table_query = """
    ALTER TABLE combined_hourly_data
    ALTER COLUMN hour TYPE TIME USING (LPAD(hour::text, 2, '0') || ':00')::time;
"""

cursor.execute(alter_table_query)


In [13]:
# Create the INSERT query with quoted column names
insert_query = sql.SQL(
    "INSERT INTO combined_hourly_data ({}) VALUES %s"
).format(
    sql.SQL(", ").join([sql.Identifier(col) for col in columns])
)


In [14]:
# Prepare data for insertion
values = [tuple(row) for row in pivot_data.to_numpy()]

In [16]:
execute_values(cursor, insert_query, values)

In [17]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [18]:
# Query to show the top 2 rows in the table
def show_top_rows():
    
    query = "SELECT * FROM combined_hourly_data LIMIT 25;"
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

# Call the function to display the top 2 rows
show_top_rows()


(datetime.date(2024, 11, 7), datetime.time(0, 0), 169.505, 167.99, 180.7, 167.38, 136.85, 277.36, 148.595, 213.565, 201.095, 160.745, 275.16, 223.765, 156.205, 232.96, 193.295, 239.42, 247.08, 188.81, 191.32, 237.505, 161.57, 201.285, 200.26, 216.76, 147.595, 194.27, 159.395, 145.73, 89.95, 151.69, 171.44, 283.115, 534.43, 171.055, 175.105, 191.195)
(datetime.date(2024, 11, 7), datetime.time(10, 0), 162.065, 116.79, 93.865, 137.38, 168.75, 178.86, 162.505, 167.375, 200.315, 177.965, 203.36, 132.775, 175.985, 157.445, 102.015, 168.935, 236.435, 134.33, 85.065, 198.84, 145.34, 113.59, 133.645, 164.345, 165.075, 139.925, 114.785, 79.625, 124.615, 237.91, 118.72, 189.58, 227.235, 185.625, 124.455, 92.865)
(datetime.date(2024, 11, 7), datetime.time(11, 0), 180.93, 114.975, 96.315, 132.65, 173.815, 182.865, 174.92, 194.325, 209.42, 178.67, 206.005, 138.875, 173.295, 161.9, 102.72, 166.21, 315.475, 139.855, 86.67, 203.645, 142.47, 124.81, 139.3, 169.335, 170.72, 140.04, 124.29, 75.05, 124.555

In [19]:
# Execute the ALTER TABLE command to add a unique constraint on date and hour
alter_table_query = """
    ALTER TABLE combined_hourly_data
    ADD CONSTRAINT unique_date_hour UNIQUE (date, hour);
"""

try:
    cursor.execute(alter_table_query)
    conn.commit()
    print("Unique constraint on (date, hour) added successfully.")
except psycopg2.Error as e:
    print("An error occurred:", e)
finally:
    cursor.close()
    conn.close()


An error occurred: relation "unique_date_hour" already exists



In [25]:
def get_last_24_hours(district):
    # Establish the connection
    conn = psycopg2.connect(**db_params)
    conn.autocommit = True
    cursor = conn.cursor()

    query = sql.SQL("""
        SELECT date, hour, {}
        FROM combined_hourly_data
        WHERE {} IS NOT NULL
        ORDER BY date DESC, hour DESC
        LIMIT 24
    """).format(sql.Identifier(district), sql.Identifier(district))

    cursor.execute(query)
    rows = cursor.fetchall()

    # Extract only the hourly values into a single array
    hourly_values = [row[2] for row in rows]
    
    # Close the connection
    cursor.close()
    conn.close()

    return hourly_values

# Example usage
get_last_24_hours('Attock')


[94.89500000000001,
 115.125,
 110.6,
 100.985,
 95.665,
 93.895,
 94.955,
 97.44,
 99.77,
 101.935,
 102.295,
 100.685,
 98.46,
 96.89,
 98.0,
 100.655,
 99.53,
 91.975,
 89.59,
 91.945,
 94.245,
 93.98,
 94.245,
 93.07]

In [27]:
import datetime


In [43]:
def fetch_hourly_aqi_from_db(district: str, date: str, db_params=db_params):
    """
    Query the database for hourly AQI data for a specified district and date.
    
    Args:
        district: Name of the district (column name for AQI data).
        date: Date in MM/DD/YYYY format.
        db_params: Dictionary containing database connection parameters.
    
    Returns:
        Tuple of hours and AQI values.
    """
    try:
        # Convert date string to date object
        query_date = datetime.datetime.strptime(date, "%m/%d/%Y").date()
        
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        conn.autocommit = True
        cursor = conn.cursor()

        # Construct the SQL query
        query = sql.SQL("""
            SELECT hour, {}
            FROM combined_hourly_data
            WHERE date = %s
            ORDER BY hour
        """).format(sql.Identifier(district))  # Dynamically select the column based on district
        
        # Execute the query with date as the parameter
        cursor.execute(query, (query_date,))
        rows = cursor.fetchall()
        
        # Close the connection
        cursor.close()
        conn.close()

        # Format hours as strings like "0:00", "1:00", etc., and extract AQI values
        if rows:
            hours = [f"{row[0].hour}:00" for row in rows]  # Format hours as strings
            aqi_values = [row[1] for row in rows]           # Extract AQI values for the district column
            return hours, aqi_values
        else:
            return None, None

    except Exception as e:
        raise Exception(f"Database query failed: {e}")

In [44]:
print(fetch_hourly_aqi_from_db("Attock", "11/16/2024"))

(['0:00', '1:00', '2:00', '3:00', '4:00'], [95.665, 100.985, 110.6, 115.125, 94.89500000000001])
